In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/melon_song.csv')
df.head(1)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6247 entries, 0 to 6246
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   songId     6247 non-null   int64  
 1   title      6247 non-null   object 
 2   artist     6247 non-null   object 
 3   genre      6245 non-null   object 
 4   album      6247 non-null   object 
 5   lyricist   5849 non-null   object 
 6   composer   5845 non-null   object 
 7   date       6202 non-null   float64
 8   img        6247 non-null   object 
 9   comment    6247 non-null   int64  
 10  like       6247 non-null   int64  
 11  lyric      6247 non-null   object 
 12  plylstSeq  6126 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 634.6+ KB


In [4]:
#  1. 컨텐츠기반 추천
#    Input(title, artist) : contains 로 찾기 => 결과 songId 한 개만 반환하도록 한다.
#    songId로 추천시스템에서 비슷한 노래 5개 찾아서 songId로 반환
#    반환된 songId 들의 정보 출력

# 가사에서 \r -> replace하기

In [4]:
df.date.fillna(0, inplace=True)
df.date = df.date.astype(int).astype(str)
df.songId = df.songId.astype(str)
df.fillna('', inplace=True)

In [7]:
# df[df.title.str.contains('희재') & df.artist.str.contains('성시경')]

In [8]:
# title, artist = map(input().split())

# df[df.title.str.contains('title') & df.artist.str.contains('artist')]

In [9]:
def songid_find(title, artist):
    ss = df[df.title.str.contains(title) & df.artist.str.contains(artist)]
    return ss.songId

In [10]:
songid_find('희재','성시경')

0     418168
55    418200
Name: songId, dtype: object

In [6]:
# 한글 불용어 처리
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [7]:
from konlpy.tag import Okt
okt = Okt()

In [12]:
df.lyric[0]

'햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞에서 \n그대 나를 떠나 간다해도 난 \n그댈 보낸적 없죠 \n기다림으로 다시 시작일테니 \n얼마나 사랑했는지 얼마나 \n또 울었는지 \n그대여 한순간 조차 잊지 말아요 \n거기 떠나간 그곳에서 날\n기억하며 기다려요\n하루씩 그대에게 다가가는 \n나 일테니'

In [8]:
lyrics = []
for lyric in df.lyric:
    lyric = lyric.replace('\n', ' ')
    morphs = okt.morphs(lyric, stem=True)
    tmp = [word for word in morphs if word not in stop_words]
    lyrics.append(' '.join(tmp))
df['morphs'] = lyrics

In [9]:
df['total'] = df.morphs + df.lyric + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 2 + (' ' + df.title)
df.total[0]

'햇살 우릴 위해 내리다 바람 서 롤 감싸다 하다 우리 웃음 속 계절 오다 또 가다 바람 흔들리다 머릿결 내다 불어오다 그대 향기 예쁘다 눈 웃음 소리 모두 가 내 이다 이렇다 사랑 이렇다 행복 쉬다 하다 이렇다 웃음 이렇다 축복 내다 쉬다 올리다 없다 눈물 조차 울음 조차 닦다 못 한 나 정말로 울면 내 가 대다 보내다 준 같다 그대 떠나가다 순간 나르다 걱정 하다 무엇 해주다 없다 내 맘 앞 에서 그대 나르다 떠나다 간다 하다 난 대다 보내다 적 없다 여전하다 대다 나르다 살 하다 이율 테 이렇다 사랑 이렇다 행복 쉬다 하다 이렇다 웃음 이렇다 축복 내다 쉬다 올리다 없다 눈물 조차 울음 조차 닦다 못 한 나 정말로 울면 내 가 대다 보내다 준 같다 그대 떠나가다 순간 나르다 걱정 하다 무엇 해주다 없다 내 맘앞 에서 그대 나르다 떠나다 간다 하다 난 대다 보내다 적 없다 기다림 으로 다시 시 작 일 테 얼마나 사랑 하다 얼마나 또 울다 그대 한순간 조차 잊다 말다 거기 떠나가다 곳 에서 날 기억 하다 기다리다 하루 씩 그대 다가가다 나 이다햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞

In [10]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)
df.total[0]

'햇살 우릴 위해 내리다 바람 서 롤 감싸다 하다 우리 웃음 속 계절 오다 또 가다 바람 흔들리다 머릿결 내다 불어오다 그대 향기 예쁘다 눈 웃음 소리 모두 가 내 이다 이렇다 사랑 이렇다 행복 쉬다 하다 이렇다 웃음 이렇다 축복 내다 쉬다 올리다 없다 눈물 조차 울음 조차 닦다 못 한 나 정말로 울면 내 가 대다 보내다 준 같다 그대 떠나가다 순간 나르다 걱정 하다 무엇 해주다 없다 내 맘 앞 에서 그대 나르다 떠나다 간다 하다 난 대다 보내다 적 없다 여전하다 대다 나르다 살 하다 이율 테 이렇다 사랑 이렇다 행복 쉬다 하다 이렇다 웃음 이렇다 축복 내다 쉬다 올리다 없다 눈물 조차 울음 조차 닦다 못 한 나 정말로 울면 내 가 대다 보내다 준 같다 그대 떠나가다 순간 나르다 걱정 하다 무엇 해주다 없다 내 맘앞 에서 그대 나르다 떠나다 간다 하다 난 대다 보내다 적 없다 기다림 으로 다시 시 작 일 테 얼마나 사랑 하다 얼마나 또 울다 그대 한순간 조차 잊다 말다 거기 떠나가다 곳 에서 날 기억 하다 기다리다 하루 씩 그대 다가가다 나 이다햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(6247, 106774)

In [12]:
indices = pd.Series(df.index, index=df.songId)
indices

songId
418168         0
35609035       1
1046278        2
35609034       3
35728845       4
            ... 
33013736    6242
3080890     6243
2981597     6244
34183979    6245
31191637    6246
Length: 6247, dtype: int64

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [22]:
cosine_sim

array([[1.        , 0.01866516, 0.05685873, ..., 0.0812984 , 0.02361475,
        0.00949515],
       [0.01866516, 1.        , 0.03522961, ..., 0.01664912, 0.0074854 ,
        0.00290335],
       [0.05685873, 0.03522961, 1.        , ..., 0.02145126, 0.00939362,
        0.01317641],
       ...,
       [0.0812984 , 0.01664912, 0.02145126, ..., 1.        , 0.01884183,
        0.033759  ],
       [0.02361475, 0.0074854 , 0.00939362, ..., 0.01884183, 1.        ,
        0.01260118],
       [0.00949515, 0.00290335, 0.01317641, ..., 0.033759  , 0.01260118,
        1.        ]])

In [14]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(6).tail(5).index
    return df.songId.iloc[song_indices]

In [15]:
a = get_recommendation('35609035', cosine_sim)
a.values

array(['35609034', '4381724', '35307770', '1053842', '1925230'],
      dtype=object)

In [16]:
a.values[0]

'35609034'

In [17]:
# 유사도 1에 가까울수록 유사함
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices['32720013']])
sim_scores.sort_values(ascending=False).head(11).tail(10)

4144    0.607869
3822    0.470227
6100    0.342800
3511    0.323353
3118    0.320993
4414    0.294566
4040    0.293420
6047    0.289387
3816    0.286265
4773    0.286147
dtype: float64

In [19]:
filtered_df = df[df['songId'].isin(a)].to_dict('records')
# filtered_df.to_dict('recodes')
filtered_df

[{'songId': '35609034',
  'title': '슬픈등',
  'artist': '김호중',
  'genre': '발라드 국내영화',
  'album': '인생은 뷰티풀: 비타돌체',
  'lyricist': '김호중',
  'composer': '김호중',
  'date': '20220907',
  'img': 'https://cdnimg.melon.co.kr/cm2/album/images/110/48/906/11048906_20220906180454_500.jpg?d530d42b8ca05e5fe8c89007570a3d26/melon/resize/282/quality/80/optimize',
  'comment': 145,
  'like': 15389,
  'lyric': '누군가를 다시 만난다면\n여전히 나는 너를\n누군가를 다시 사랑하면\n영원히 나는 너를\n다시 또 누군가를 그리라면\n망설임 하나 없이 나는 너를\n하나 누군가 다시 떠나보내라면\n두 번 죽어도 너와는\n못다 한 사랑이라 부르지 마\n덧없는 미련이라 탓하지 마\n꺼지지 않는 밤 하늘\n별빛이 사라질 때까지\n외로움아 거기 서 있거라\n보내지 않았으니\n그리움아 그냥 스쳐 가라\n시작도 못 했으니\n바람이 부는 언덕길을 걷는 그녀와\n그녀의 긴 그림자가\n다정한 연인처럼 내 마음을\n지나 날 떠나가고 있지만\n못다 한 사랑이라 말하지 마\n덧없는 미련이라 탓하지 마\n꺼지지 않는 밤 하늘 별빛이\n사라질 때까지\n한 번만이라도 듣고 싶어\n그녀가 부르는 내 이름을\n한 번 더 바라보고 싶어\n날 바라보는 그 얼굴\n바람이 불어오는 언덕길을\n그녀와 그녀의 긴 그림자 되어\n단 한 번만 더 걸을 수 있다면\n죽어도 좋으리',
  'plylstSeq': '457536944 515278519',
  'total': '누군가 다시 만나다 여전하다 나 너 누군가 다시 사랑 하다 영원하다 나 너 다시 또 누군가 그리다 망설이다 하나 없이 나 너 하나 누군가 다시 떠나다 보내다 죽다 너